In [1]:
# CSC478 Final Project Itembased app
import os
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Load movieLens data
path = 'F:\Computer science\csc478\csc478 Final\movielens\Movielens-02'
os.chdir(path)
users = pd.read_table('u.user',delimiter='|',header=None,names=["id","age","gender","job","zip code"])
movies = pd.read_table('u.item',delimiter='|',header=None, names=["id","title","rl_date","vrl_date","url",
                                                                 "unknown","Action","Adventure","Animation",
                                                                 "Children's ","Comedy","Crime","Documentary",
                                                                 "Drama","Fantasy","Film-Noir","Horror",
                                                                 "Musical","Mystery","Romance","Sci-Fi",
                                                                 "Thriller","War","Western"])
ratings = pd.read_table('u.data',delimiter='\t',header=None,names=["user","movie","rating","timestamp"])
genres = pd.read_table('u.genre',delimiter='|',header=None,names=["genre","id"])

In [3]:
npmovies = np.genfromtxt('u.item',delimiter='|',dtype=str)

In [4]:
npmovies.shape

(1682L, 24L)

In [5]:
npuser = np.genfromtxt('u.user',delimiter='|',dtype=str)

In [6]:
npuser


array([['1', '24', 'M', 'technician', '85711'],
       ['2', '53', 'F', 'other', '94043'],
       ['3', '23', 'M', 'writer', '32067'],
       ..., 
       ['941', '20', 'M', 'student', '97229'],
       ['942', '48', 'F', 'librarian', '78209'],
       ['943', '22', 'M', 'student', '77841']], 
      dtype='|S13')

In [7]:
#list1=ratelist(npratings,npmovies,npuser)

In [8]:
rating_mat=np.zeros((len(users),len(movies)))
rating_mat.shape

(943L, 1682L)

In [9]:

for i in range(len(ratings)):
    user=int(ratings.user[i])-1
    movie=int(ratings.movie[i])-1
    rating=int(ratings.rating[i])
    rating_mat[user,movie]=rating

In [11]:
rating_mat.shape

(943L, 1682L)

In [13]:
import numpy as np
import pylab as pl
from sklearn .cluster import KMeans
from sklearn import preprocessing

from sklearn import metrics

In [15]:
rating_mat

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

In [172]:
npmovies

array([['1', 'Toy Story (1995)', '01-Jan-1995', ..., '0', '0', '0'],
       ['2', 'GoldenEye (1995)', '01-Jan-1995', ..., '1', '0', '0'],
       ['3', 'Four Rooms (1995)', '01-Jan-1995', ..., '1', '0', '0'],
       ..., 
       ['1680', 'Sliding Doors (1998)', '01-Jan-1998', ..., '0', '0', '0'],
       ['1681', 'You So Crazy (1994)', '01-Jan-1994', ..., '0', '0', '0'],
       ['1682', 'Scream of Stone (Schrei aus Stein) (1991)', '08-Mar-1996',
        ..., '0', '0', '0']], 
      dtype='|S134')

In [151]:
from numpy import *
from numpy import linalg as la
import numpy as np

def ecludSim(inA,inB):
    return 1.0 / (1.0 + la.norm(inA - inB))

def pearsSim(inA,inB):
    if len(inA) < 3 : return 1.0
    return 0.5 + 0.5 * corrcoef(inA, inB, rowvar = 0)[0][1]

def cosSim(inA,inB):
    num = float(inA.T * inB)
    denom = la.norm(inA)*la.norm(inB)
    return 0.5 + 0.5 * (num / denom)
def cosSim2(vecA,vecB):
    return np.dot(vecA, vecB) / ( np.linalg.norm(vecA) * np.linalg.norm(vecB) )

def standEst(dataMat, user, simMeas, item):
    n = shape(dataMat)[1]
    simTotal = 0.0; ratSimTotal = 0.0
    for j in range(n):
        userRating = dataMat[user,j]
        if userRating == 0: continue
        overLap = nonzero(logical_and(dataMat[:,item]>0, \
                                      dataMat[:,j]>0))[0]
        if len(overLap) == 0: similarity = 0
        else: similarity = simMeas(dataMat[overLap,item], \
                                   dataMat[overLap,j])
        #print 'the %d and %d similarity is: %f' % (item, j, similarity)
        simTotal += similarity
        ratSimTotal += similarity * userRating
    if simTotal == 0: return 0
    else: return ratSimTotal/simTotal
    




# This function performs cross-validation on a single user based on the test_ratio
# For example, with test_ratio = 0.2, 5-fold x-validation is performed where in each fold, 
# 20 percent of rated items are withheld and the rest are used to estimate the withheld ratings

def cross_validate_user(dataMat, user, test_ratio, estMethod=standEst, simMeas=pearsSim):
    number_of_items = np.shape(dataMat)[1]
    rated_items_by_user = np.array([i for i in range(number_of_items) if dataMat[user,i]>0])
    test_size = test_ratio * len(rated_items_by_user)
    test_indices = np.random.randint(0, len(rated_items_by_user), test_size)
    withheld_items = rated_items_by_user[test_indices]
    original_user_profile = np.copy(dataMat[user])
    dataMat[user, withheld_items] = 0 # So that the withheld test items is not used in the rating estimation below
    error_u = 0.0
    count_u = len(withheld_items)
# Compute absolute error for user u over all test items
    for item in withheld_items:
    # Estimate rating on the withheld item
        estimatedScore = estMethod(dataMat, user, simMeas, item)
        error_u = error_u + abs(estimatedScore - original_user_profile[item])	

# Now restore ratings of the withheld items to the user profile
    for item in withheld_items:
        dataMat[user, item] = original_user_profile[item]

    # Return sum of absolute errors and the count of test cases for this user
    # Note that these will have to be accumulated for each user to compute MAE
    return error_u, count_u



def test(MData, testRatio, method,simMethod):
    totalerror = 0.0
    total = 0
    for user in range(len(MData)):
        
        error, count = cross_validate_user(MData, user, testRatio, estMethod=method,simMeas=simMethod)
        
        totalerror += error
        total += count
    MAE = totalerror / float(total)
    print 'Mean Absoloute Error for ',method,' : ', MAE

def test1(MData, testRatio, method,simMethod):
    totalerror = 0.0
    total = 0
    for user in range(len(MData)):
        
        error, count = cross_validate_user(MData, user, testRatio, estMethod=method,simMeas=simMethod)
        if isnan(error):
            continue
        totalerror += error
        total += count
    MAE = totalerror / float(total)
    print 'Mean Absoloute Error for ',method,' : ', MAE
    


def get_mv_text(movie, id):
    return movie[id]


In [ ]:
# Mean absoloute for cosine simalarity

In [129]:
test(rating_mat, 0.2, standEst,cosSim2)


Mean Absoloute Error for  <function standEst at 0x000000000D952DD8>  :  0.829079195273


e:\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
#MAE for ecludSim

In [131]:
test(rating_mat, 0.2, standEst,ecludSim)

Mean Absoloute Error for  <function standEst at 0x000000000D952DD8>  :  0.834439256357


e:\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [ ]:
#MAE for pearsSim

In [150]:
test1(rating_mat,0.2,standEst,pearsSim)

Mean Absoloute Error for  <function standEst at 0x000000000D952DD8>  :  0.816964270507


e:\Anaconda2\lib\site-packages\ipykernel\__main__.py:69: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [168]:
def rec_most_similar_movies(dataMat, movie, querymovie, k, metric=pearsSim):
    mv = get_mv_text(movie, querymovie)
    print 'Selected Movie: \n'
    print mv[:5]
    
    simMovie = zeros(dataMat.shape[1])
    for i in range(dataMat.shape[1]):
        if i != querymovie:
            simMovie[i] = metric(dataMat[:,i], dataMat[:, querymovie])
    sortedVec = simMovie.argsort()[::-1]
    print '\nTop 5 Recommended movies are :\n'
    for i in range(k):
        print movie[sortedVec[i]][:5], '\n_______________\n'

In [169]:
# Count Knn by pearsSim, then recommand the most sim n.
# The sim is based on the movoe's rating

In [170]:
rec_most_similar_movies(rating_mat,npmovies,3,5,pearsSim)

Selected Movie: 

['4' 'Get Shorty (1995)' '01-Jan-1995' ''
 'http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)']

Top 5 Recommended movies are :

['56' 'Pulp Fiction (1994)' '01-Jan-1994' ''
 'http://us.imdb.com/M/title-exact?Pulp%20Fiction%20(1994)'] 
_______________

['202' 'Groundhog Day (1993)' '01-Jan-1993' ''
 'http://us.imdb.com/M/title-exact?Groundhog%20Day%20(1993)'] 
_______________

['204' 'Back to the Future (1985)' '01-Jan-1985' ''
 'http://us.imdb.com/M/title-exact?Back%20to%20the%20Future%20(1985)'] 
_______________

['385' 'True Lies (1994)' '01-Jan-1994' ''
 'http://us.imdb.com/M/title-exact?True%20Lies%20(1994)'] 
_______________

['96' 'Terminator 2: Judgment Day (1991)' '01-Jan-1991' ''
 'http://us.imdb.com/M/title-exact?Terminator%202:%20Judgment%20Day%20(1991)'] 
_______________



In [20]:
def unratedMovies(userId,movies,datamat):
    unrated = []
    for i in range(len(movies)):
        if datamat[userId,i] ==0:
            unrated.append(i)
    return unrated

In [21]:
# Predict all unrated movie for a user
# predict based by that user 's rating on most similiar movies
def PredictRateForUser(dataMat, movie, userId, metric=pearsSim):
    #mv = get_mv_text(movie, querymovie)
    #print 'Selected Movie: \n'
    #print mv[:5]
    
    unrated = unratedMovies(userId,movies,dataMat)
    simMovie = zeros(dataMat.shape[1])
    for querymovie in unrated:
        
        for i in range(dataMat.shape[1]):
            if i != querymovie:
                simMovie[i] = metric(dataMat[:,i], dataMat[:, querymovie])
        sortedVec = simMovie.argsort()[::-1]
        for j in sortedVec:
            if dataMat[userId][j] !=0:
                t = dataMat[userId][j]
                break
        print "Predict rating for user # %d movie: %s is: %d "%(userId,movie[querymovie][1],t)
    
   
        
        
        
    

In [165]:
 PredictRateForUser(rating_mat,npmovies,1,pearsSim)

Predict rating for user # 1 movie: GoldenEye (1995) is: 5 
Predict rating for user # 1 movie: Four Rooms (1995) is: 4 
Predict rating for user # 1 movie: Get Shorty (1995) is: 5 
Predict rating for user # 1 movie: Copycat (1995) is: 3 
Predict rating for user # 1 movie: Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) is: 5 
Predict rating for user # 1 movie: Twelve Monkeys (1995) is: 5 
Predict rating for user # 1 movie: Babe (1995) is: 5 
Predict rating for user # 1 movie: Dead Man Walking (1995) is: 4 
Predict rating for user # 1 movie: Seven (Se7en) (1995) is: 5 
Predict rating for user # 1 movie: Usual Suspects, The (1995) is: 5 
Predict rating for user # 1 movie: Mr. Holland's Opus (1995) is: 4 
Predict rating for user # 1 movie: French Twist (Gazon maudit) (1995) is: 5 
Predict rating for user # 1 movie: From Dusk Till Dawn (1996) is: 4 
Predict rating for user # 1 movie: White Balloon, The (1995) is: 5 
Predict rating for user # 1 movie: Angels and Insects (1995) is: 5 
Pre

KeyboardInterrupt: 

In [24]:
# Predict all unrated movie for a user and a specific movie
def PredictRateForUserMovie(dataMat, movie, userId,querymovie,metric=pearsSim):
    #mv = get_mv_text(movie, querymovie)
    #print 'Selected Movie: \n'
    #print mv[:5]
    
    unrated = unratedMovies(userId,movies,dataMat)
    simMovie = zeros(dataMat.shape[1])
    if querymovie not in unrated:
        print "User # %d has rated the movie, the rating is %d "%(userId,dataMat[userId][querymovie])
    else:   
        for i in range(dataMat.shape[1]):
            if i != querymovie:
                simMovie[i] = metric(dataMat[:,i], dataMat[:, querymovie])
        sortedVec = simMovie.argsort()[::-1]
        for j in sortedVec:
            if dataMat[userId][j] !=0:
                t = dataMat[userId][j]
                break
        print "Predict rating for user # %d, movie:  %s is: %d "%(userId,movie[querymovie][1],t)
    
   
        

In [25]:
PredictRateForUserMovie(rating_mat,npmovies,1,2,pearsSim)

Predict rating for user # 1, movie:  Four Rooms (1995) is: 4 


In [ ]:
# PredictTest will predict rating for a rated movie of the user, and then compare the predict result with actual rating 

In [26]:
def PredictTest(dataMat, movie, userId, metric=pearsSim):
    #mv = get_mv_text(movie, querymovie)
    #print 'Selected Movie: \n'
    #print mv[:5]
    
    unrated = unratedMovies(userId,movies,dataMat)
    simMovie = zeros(dataMat.shape[1])
    
    for j in range(dataMat.shape[1]) :
        if j not in unrated:
            for i in range(dataMat.shape[1]):
                if i not in unrated and i !=j:
                    simMovie[i] = metric(dataMat[:,i], dataMat[:, j])
            sortedVec = simMovie.argsort()[::-1]
            for k in sortedVec:
                if dataMat[userId][k] !=0:
                    t = dataMat[userId][k]
                    break
            print "Predict rating for user # %d movie: %s is: %d "%(userId,movie[j][1],t)
            print "Actual rating for user # %d movie: %s is %d"%(userId,movie[j][1],dataMat[userId][j])
            print ""

In [286]:
 PredictTest(rating_mat,npmovies,1,pearsSim)

Predict rating for user # 1 movie: Toy Story (1995) is: 5 
Actual rating for user # 1 movie: Toy Story (1995) is 4

Predict rating for user # 1 movie: Richard III (1995) is: 4 
Actual rating for user # 1 movie: Richard III (1995) is 2

Predict rating for user # 1 movie: Mighty Aphrodite (1995) is: 5 
Actual rating for user # 1 movie: Mighty Aphrodite (1995) is 4

Predict rating for user # 1 movie: Postino, Il (1994) is: 5 
Actual rating for user # 1 movie: Postino, Il (1994) is 4

Predict rating for user # 1 movie: Antonia's Line (1995) is: 3 
Actual rating for user # 1 movie: Antonia's Line (1995) is 3

Predict rating for user # 1 movie: Birdcage, The (1996) is: 4 
Actual rating for user # 1 movie: Birdcage, The (1996) is 4

Predict rating for user # 1 movie: Star Wars (1977) is: 4 
Actual rating for user # 1 movie: Star Wars (1977) is 5

Predict rating for user # 1 movie: Fargo (1996) is: 4 
Actual rating for user # 1 movie: Fargo (1996) is 5

Predict rating for user # 1 movie: Truth

In [ ]:
#PredictTest2 will find the the accuracy by how close the predicted movie rating to the actual movie rating.

In [56]:
def PredictTest2(dataMat, movie, userId, metric=pearsSim):
    #mv = get_mv_text(movie, querymovie)
    #print 'Selected Movie: \n'
    #print mv[:5]
    
    unrated = unratedMovies(userId,movies,dataMat)
    simMovie = zeros(dataMat.shape[1])
    n=0
    p=0
    for j in range(dataMat.shape[1]) :
        if j not in unrated:
            for i in range(dataMat.shape[1]):
                if i not in unrated and i !=j:
                    simMovie[i] = metric(dataMat[:,i], dataMat[:, j])
            sortedVec = simMovie.argsort()[::-1]
            for k in sortedVec:
                if dataMat[userId][k] !=0:
                    t = dataMat[userId][k]
                    break
            p +=t/dataMat[userId][j] if t < dataMat[userId][j] else dataMat[userId][j]/t
            
            n+=1
           
    print "acuracy is : %f"%(p/n)

In [ ]:
#accury using pearsSim

In [167]:
PredictTest2(rating_mat,npmovies,i,pearsSim)

acuracy is : 0.761582


In [ ]:
#PRedictTest4 is modified edition of PredictTest2, it returns the accurcy for a single user instead of printing it out.

In [158]:
def PredictTest4(dataMat, movie, userId, metric=pearsSim):
    #mv = get_mv_text(movie, querymovie)
    #print 'Selected Movie: \n'
    #print mv[:5]
    
    unrated = unratedMovies(userId,movies,dataMat)
    simMovie = zeros(dataMat.shape[1])
    n=0
    p=0
    for j in range(dataMat.shape[1]) :
        if j not in unrated:
            for i in range(dataMat.shape[1]):
                if i not in unrated and i !=j:
                    simMovie[i] = metric(dataMat[:,i], dataMat[:, j])
            sortedVec = simMovie.argsort()[::-1]
            for k in sortedVec:
                if dataMat[userId][k] !=0:
                    t = dataMat[userId][k]
                    break
            p +=t/dataMat[userId][j] if t < dataMat[userId][j] else dataMat[userId][j]/t
            
            n+=1
            t= p/n
    return t

In [ ]:
#find 20% of the total user data

In [159]:
len(rating_mat)*0.2

188.60000000000002

In [ ]:
#So I would like to use 189 user as test data

In [ ]:
#accury using ecludSim

In [162]:
n=0
k=0
for i in range(1,190):
    k+=PredictTest4(rating_mat,npmovies,i,ecludSim)
    n+=1
print "The accuracy using ecludSim is %f"%(k/n)

The accuracy using ecludSim is 0.775267


In [ ]:
#accury using pearsSim

In [163]:
n=0
k=0
for i in range(1,190):
    k+=PredictTest4(rating_mat,npmovies,i,pearsSim)
    n+=1
print "The accuracy using pearsSim is %f"%(k/n)

The accuracy using pearsSim is 0.796405


In [104]:
#accurcy using CosSim

In [164]:
n=0
k=0
for i in range(1,190):
    k+=PredictTest4(rating_mat,npmovies,i,cosSim2)
    n+=1
print "The accuracy using cosSim is %f"%(k/n)

The accuracy using cosSim is 0.795872
